# Compute Equatorial Upwelling Current from ECCO version 4 release 4

In [1]:
from glob import glob
import warnings
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from xgcm import Grid
import ecco_v4_py as ecco
import pych.ecco as pe

In [2]:
uvel_download_dir = '/workspace/ecco/release4/nctiles_monthly/UVELMASS/*/*.nc'
vvel_download_dir = '/workspace/ecco/release4/nctiles_monthly/VVELMASS/*/*.nc'

In [3]:
grid_dir = '/workspace/grids/llc90/'

In [4]:
ds = xr.open_mfdataset(glob(uvel_download_dir)+glob(vvel_download_dir))

In [5]:
g = pe.read_mds(grid_dir,iters=None)

/workspace/xmitgcm/xmitgcm/mds_store.py:850: UserWarning: Couldn't find available_diagnostics.log in /workspace/grids/llc90/ or /workspace/grids/llc90/. Using default version.
  "in %s or %s. Using default version." % (data_dir, grid_dir))


## Create mask defining the Equatorial Upwelling Current (EUC)

Consider the region $140^\circ$E-$80^\circ$W, $1.5^\circ$S-$1.5^\circ$N, from the surface to 400m depth.

In [6]:
lon_left = np.arange(140,180,1)
lon_right = np.arange(-180,-79.5,1)

In [7]:
lon_list = np.concatenate((lon_left,lon_right))

In [8]:
for fld in ['drF','dyG','dxG','XC']:
    ds[fld] = g[fld]
    ds = ds.set_coords(fld)

In [9]:
ds['UVELMASS'] = ds.UVELMASS.where(ds.Z>-400)
ds['VVELMASS'] = ds.VVELMASS.where(ds.Z>-400)

### Load some often used coordinates to memory

In [10]:
ds['dyG'].load();
ds['dxG'].load();
ds['drF'].load();
ds['XC'].load();
ds['YC'].load();

In [11]:
grid = ecco.get_llc_grid(ds)

In [12]:
def calc_euc(ds,pt1,pt2,grid):
    
    maskW,maskS = ecco.calc_section_trsp._parse_section_trsp_inputs(ds,pt1,pt2,None,None,'EUC')
    
    # Get surface area for transport
    area_x = ds['drF']*ds['dyG']*maskW
    area_y = ds['drF']*ds['dxG']*maskS
    
    # Get volumetric transport at grid cell
    xvol = ds['UVELMASS']*area_x
    yvol = ds['VVELMASS']*area_y
    
    # Compute cross sectional area as a function of time
    area_x_t = xr.where(ds['UVELMASS']!=0,xvol/ds['UVELMASS'],0.)
    area_y_t = xr.where(ds['VVELMASS']!=0,yvol/ds['VVELMASS'],0.)
    
    # initialize ds
    trsp_ds = ecco.calc_section_trsp._initialize_section_trsp_data_array(ds)
    
    # Integrate, only consider U>0
    # Note: I know this condition is OK because after multiplying by the masks above,
    # both xvol and yvol should be >0 for U>0 on the tiles in consideration here
    trsp_x = xvol.where(xvol>0).sum(dim=['i_g','j','tile','k'])
    trsp_y = yvol.where(yvol>0).sum(dim=['i','j_g','tile','k'])
    trsp_ds['trsp'] = (trsp_x + trsp_y)* (10**-6)
    trsp_ds['trsp'].attrs['units'] = 'Sv'
    trsp_ds['maskW'] = maskW
    trsp_ds['maskS'] = maskS
    
    return trsp_ds

### Make a container dataset for EUC transport as a function of time and longitude

with field 'trsp' as zeros for now.

In [13]:
euc=xr.Dataset()
euc['lon'] = np.array(lon_list)
euc['time'] = ds['time']

In [14]:
for fld in ['trsp']:
    euc[fld] = xr.zeros_like(xr.broadcast(euc.time,euc.lon)[0],dtype=np.dtype('float64'))
    euc[fld].attrs={}

In [15]:
ds['UVELMASS'] = ds.UVELMASS.chunk((12,50,13,90,90))
ds['VVELMASS'] = ds.VVELMASS.chunk((12,50,13,90,90))

In [ ]:
%%time
for i,lon in enumerate(lon_list):
    pt1 = [lon,1.5]
    pt2 = [lon,-1.5]
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore')
        trsp_ds = calc_euc(ds,pt1=pt1,pt2=pt2,grid=grid)
        for fld in ['trsp']:
            euc[fld].loc[{'lon':lon}] = trsp_ds[fld].values
            euc[fld].attrs = trsp_ds[fld].attrs
    print(f' --- Done: {i}/{len(lon_list)} --- ')

In [ ]:
euc.to_netcdf('/workspace/results/eccov4r4/equatorial-upwelling-current/euc_eccov4r4.nc')

In [ ]:
lon_index = np.arange(len(euc.lon))

In [ ]:
lon_label_e = [f'{int(l)}$^\circ$E' for l in euc.lon.where(euc.lon>0,drop=True).values]
lon_label_w = [f'{-int(l)}$^\circ$W' for l in euc.lon.where(euc.lon<0,drop=True).values]

In [ ]:
lon_label = lon_label_e+lon_label_w

## For plot, select 1993-2017

In [ ]:
euc_mean = euc['trsp'].sel(time=slice('1993','2017')).mean(dim='time')
euc_std = euc['trsp'].sel(time=slice('1993','2017')).std(dim='time')

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(15,6))

ax.plot(lon_index,euc_mean,'k')
ax.fill_between(lon_index,euc_mean-euc_std,euc_mean+euc_std,alpha=.3,color='gray')
plt.grid();
dx = 15
ax.set_xticks(lon_index[slice(0,None,dx)])
ax.set_xticklabels(lon_label[slice(0,None,dx)]);
ax.set_ylabel('EUC Transport [Sv]')
ax.set_title('EUC Transport from ECCOv4r4, 1993-2017\ntime mean $\pm$ standard deviation [Sv]')
ax.set_xlabel('Longitude');
fig.savefig('/workspace/results/eccov4r4/equatorial-upwelling-current/euc_timemean_eccov4r4.png',bbox_inches='tight',dpi=300)

## Make dataset nice and save to a couple different formats

In [ ]:
euc['lon_index'] = lon_index
euc['lon_label'] = lon_label

In [ ]:
euc['trsp'].attrs['long_name'] = 'Equatorial Under Current (EUC)'
euc['trsp'].attrs['Definition'] = 'Monthly (positive only, eastward) zonal velocity integrated 0-400m depth, 1.5S to 1.5N'
euc.attrs['Description'] = 'EUC from ECCOv4r4, computed by Tim Smith'

In [ ]:
euc.to_netcdf('/workspace/results/eccov4r4/equatorial-upwelling-current/euc_eccov4r4.nc')

In [ ]:
df = euc['trsp'].sel(time=slice('1993','2017')).mean('time').to_dataframe()

In [ ]:
df.to_csv('/workspace/results/eccov4r4/equatorial-upwelling-current/euc_eccov4r4_timemean_1993-2017.csv')